In [10]:
import pandas as pd
df = pd.read_pickle("team_df.pkl")

In [11]:
slot_cols = [
    'ally_ban0', 'ally_ban1', 'ally_ban2', 'ally_ban3', 'ally_ban4',
    'enemy_ban0', 'enemy_ban1', 'enemy_ban2', 'enemy_ban3', 'enemy_ban4',
    'ally_TOP', 'ally_JUNGLE', 'ally_MIDDLE', 'ally_BOTTOM', 'ally_UTILITY',
    'enemy_TOP', 'enemy_JUNGLE', 'enemy_MIDDLE', 'enemy_BOTTOM', 'enemy_UTILITY',
]

In [12]:
champ_cols = [
    'ally_BOTTOM', 'ally_JUNGLE', 'ally_MIDDLE', 'ally_TOP', 'ally_UTILITY',
    'enemy_TOP', 'enemy_JUNGLE', 'enemy_MIDDLE', 'enemy_BOTTOM', 'enemy_UTILITY',
    'ally_ban0', 'ally_ban1', 'ally_ban2', 'ally_ban3', 'ally_ban4',
    'enemy_ban0', 'enemy_ban1', 'enemy_ban2', 'enemy_ban3', 'enemy_ban4',
]

# Build champion vocabulary
all_champs = pd.unique(df[champ_cols].values.ravel())
champ2id = {c: i for i, c in enumerate(all_champs)}
id2champ = {i: c for c, i in champ2id.items()}

MASK_ID = len(champ2id)
PAD_ID  = len(champ2id) + 1
vocab_size = len(champ2id) + 2

In [ ]:
import torch
from torch.utils.data import Dataset

class DraftDataset(Dataset):
    def __init__(self, df, champ2id, slot_cols, role_slot_name='ally_TOP'):
        self.df = df.reset_index(drop=True)
        self.champ2id = champ2id
        self.slot_cols = slot_cols
        self.role_slot_idx = slot_cols.index(role_slot_name)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Sequence of champs in fixed slot order
        champs = [row[c] for c in self.slot_cols]
        seq = torch.tensor([self.champ2id[c] for c in champs], dtype=torch.long)

        # Target is the champ in the role slot to predict
        target = seq[self.role_slot_idx].clone()

        # Mask that slot in the input
        masked_seq = seq.clone()
        masked_seq[self.role_slot_idx] = MASK_ID
        slot_ids = torch.arange(len(self.slot_cols), dtype=torch.long)

        return {
            "input_ids": masked_seq,   # [L]
            "slot_ids": slot_ids,      # [L]
            "target": target,          # scalar
        }


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DraftSlotTransformer(nn.Module):
    def __init__(
        self,
        num_champs: int,
        num_slots: int = 20,
        d_model: int = 256,
        n_heads: int = 8,
        n_layers: int = 4,
        dim_ff: int = 512,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.num_champs = num_champs
        self.num_slots = num_slots
        self.d_model = d_model

        # Embeddings
        self.champ_embed = nn.Embedding(num_champs, d_model)
        self.slot_embed  = nn.Embedding(num_slots, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=dim_ff,
            dropout=dropout,
            batch_first=True,   # [B, L, D]
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        # Head to predict champ for a specific slot
        self.head = nn.Linear(d_model, num_champs)

    def forward(self, input_ids, slot_ids, target_slot_idx):
        """
        input_ids:    [B, L] champ ids (with MASK_ID at target slot)
        slot_ids:     [B, L] slot indices 0..L-1
        target_slot_idx: int (which slot we're predicting, e.g. ally_TOP index)
        """
        x = self.champ_embed(input_ids) + self.slot_embed(slot_ids)  # [B, L, D]
        h = self.encoder(x)                                         # [B, L, D]

        # Hidden rep at the target slot
        B, L, D = h.shape
        target_pos = torch.full((B,), target_slot_idx, dtype=torch.long, device=h.device)
        h_target = h[torch.arange(B, device=h.device), target_pos]  # [B, D]

        logits = self.head(h_target)  # [B, num_champs]
        return logits


In [15]:
def training_step(model, batch, target_slot_idx):
    input_ids = batch["input_ids"].to(model.champ_embed.weight.device)  # [B, L]
    slot_ids  = batch["slot_ids"].to(model.champ_embed.weight.device)   # [B, L]
    targets   = batch["target"].to(model.champ_embed.weight.device)     # [B]

    logits = model(input_ids, slot_ids, target_slot_idx)  # [B, V]
    loss = F.cross_entropy(logits, targets)
    return loss


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import AdamW
from tqdm import tqdm

# basic device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
df = pd.read_pickle("team_df.pkl")


champ_cols = [
    'ally_BOTTOM', 'ally_JUNGLE', 'ally_MIDDLE', 'ally_TOP', 'ally_UTILITY',
    'enemy_TOP', 'enemy_JUNGLE', 'enemy_MIDDLE', 'enemy_BOTTOM', 'enemy_UTILITY',
    'ally_ban0', 'ally_ban1', 'ally_ban2', 'ally_ban3', 'ally_ban4',
    'enemy_ban0', 'enemy_ban1', 'enemy_ban2', 'enemy_ban3', 'enemy_ban4',
]

# Get all champs, drop NaNs
all_champs = pd.unique(df[champ_cols].values.ravel())
all_champs = [c for c in all_champs if pd.notna(c)]
all_champs = np.sort(all_champs)

champ2id = {c: i for i, c in enumerate(all_champs)}
id2champ = {i: c for c, i in champ2id.items()}

MASK_ID = len(champ2id)       # special "to predict" token
PAD_ID  = len(champ2id) + 1   # special "empty/no champ" token
vocab_size = len(champ2id) + 2

print("Vocab size (with MASK + PAD):", vocab_size)


Using device: cuda
Vocab size (with MASK + PAD): 173


In [ ]:
slot_cols = [
    'ally_ban0', 'ally_ban1', 'ally_ban2', 'ally_ban3', 'ally_ban4',
    'enemy_ban0', 'enemy_ban1', 'enemy_ban2', 'enemy_ban3', 'enemy_ban4',
    'ally_TOP', 'ally_JUNGLE', 'ally_MIDDLE', 'ally_BOTTOM', 'ally_UTILITY',
    'enemy_TOP', 'enemy_JUNGLE', 'enemy_MIDDLE', 'enemy_BOTTOM', 'enemy_UTILITY',
]

class DraftDataset(Dataset):
    def __init__(self, df, champ2id, slot_cols, role_slot_name, mask_id, pad_id):
        """
        role_slot_name: which slot we are predicting (e.g. 'ally_TOP')
        We mask that slot in input and use the original champ as target.
        """
        self.slot_cols = slot_cols
        self.champ2id = champ2id
        self.mask_id = mask_id
        self.pad_id = pad_id
        self.role_slot_idx = slot_cols.index(role_slot_name)

        # Filter out rows where the target slot is NaN (no champ to predict)
        df = df[df[role_slot_name].notna()].copy()
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        champs = [row[c] for c in self.slot_cols]

        # Map champs to IDs, using PAD_ID for NaNs
        ids = []
        for c in champs:
            if pd.isna(c):
                ids.append(self.pad_id)
            else:
                ids.append(self.champ2id[c])
        seq = torch.tensor(ids, dtype=torch.long)

        # Target = champ in the role slot
        target = seq[self.role_slot_idx].clone()

        # Mask that slot in the input sequence
        masked_seq = seq.clone()
        masked_seq[self.role_slot_idx] = self.mask_id

        # slot ids
        L = len(self.slot_cols)
        slot_ids = torch.arange(L, dtype=torch.long)

        return {
            "input_ids": masked_seq,  # [L]
            "slot_ids": slot_ids,     # [L]
            "target": target,         # scalar
        }


In [ ]:
class DraftSlotTransformer(nn.Module):
    def __init__(
        self,
        num_champs: int,
        num_slots: int = 20,
        d_model: int = 256,
        n_heads: int = 8,
        n_layers: int = 4,
        dim_ff: int = 512,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.num_champs = num_champs
        self.num_slots = num_slots
        self.d_model = d_model

        self.champ_embed = nn.Embedding(num_champs, d_model)
        self.slot_embed  = nn.Embedding(num_slots, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=dim_ff,
            dropout=dropout,
            batch_first=True,  # [B, L, D]
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.head = nn.Linear(d_model, num_champs)

    def forward(self, input_ids, slot_ids, target_slot_idx):
        """
        input_ids:      [B, L] champ ids with MASK_ID at target slot
        slot_ids:       [B, L] slot indices 0..L-1
        target_slot_idx: int (same for the whole batch)
        """
        x = self.champ_embed(input_ids) + self.slot_embed(slot_ids)  # [B, L, D]
        h = self.encoder(x)                                          # [B, L, D]

        B, L, D = h.shape
        target_pos = torch.full((B,), target_slot_idx, dtype=torch.long, device=h.device)
        h_target = h[torch.arange(B, device=h.device), target_pos]   # [B, D]

        logits = self.head(h_target)  # [B, num_champs]
        return logits


In [ ]:
def train_one_epoch(model, loader, optimizer, target_slot_idx):
    model.train()
    total_loss = 0.0
    total_samples = 0

    for batch in tqdm(loader, desc="Train", leave=False):
        input_ids = batch["input_ids"].to(device)  # [B, L]
        slot_ids  = batch["slot_ids"].to(device)   # [B, L]
        targets   = batch["target"].to(device)     # [B]

        optimizer.zero_grad()
        logits = model(input_ids, slot_ids, target_slot_idx)  # [B, V]
        loss = F.cross_entropy(logits, targets)
        loss.backward()
        optimizer.step()

        batch_size = targets.size(0)
        total_loss += loss.item() * batch_size
        total_samples += batch_size

    return total_loss / total_samples


@torch.no_grad()
def evaluate(model, loader, target_slot_idx, topk_list=(1, 3, 5)):
    model.eval()
    total_loss = 0.0
    total_samples = 0

    # counts for each k
    correct_at_k = {k: 0 for k in topk_list}

    for batch in tqdm(loader, desc="Eval", leave=False):
        input_ids = batch["input_ids"].to(device)  # [B, L]
        slot_ids  = batch["slot_ids"].to(device)   # [B, L]
        targets   = batch["target"].to(device)     # [B]

        logits = model(input_ids, slot_ids, target_slot_idx)  # [B, V]
        loss = F.cross_entropy(logits, targets)

        batch_size = targets.size(0)
        total_loss += loss.item() * batch_size
        total_samples += batch_size

        probs = F.softmax(logits, dim=-1)  # [B, V]

        for k in topk_list:
            topk = torch.topk(probs, k=k, dim=-1).indices  # [B, k]
            # check if target in top-k
            match = (topk == targets.unsqueeze(-1)).any(dim=-1)  # [B]
            correct_at_k[k] += match.sum().item()

    avg_loss = total_loss / total_samples
    metrics = {"loss": avg_loss}
    for k in topk_list:
        metrics[f"top{k}_acc"] = correct_at_k[k] / total_samples

    return metrics


In [ ]:
ally_roles = ['ally_TOP', 'ally_JUNGLE', 'ally_MIDDLE', 'ally_BOTTOM', 'ally_UTILITY']

num_epochs = 10
batch_size = 128
val_frac = 0.2

results = {}

for role_slot_name in ally_roles:
    print(f"\n=== Training model for role: {role_slot_name} ===")

    dataset = DraftDataset(
        df=df,
        champ2id=champ2id,
        slot_cols=slot_cols,
        role_slot_name=role_slot_name,
        mask_id=MASK_ID,
        pad_id=PAD_ID,
    )

    # train/val split
    total_len = len(dataset)
    val_len = int(total_len * val_frac)
    train_len = total_len - val_len

    train_ds, val_ds = random_split(dataset, [train_len, val_len])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, drop_last=False)

    # model
    model = DraftSlotTransformer(
        num_champs=vocab_size,
        num_slots=len(slot_cols),
        d_model=256,
        n_heads=8,
        n_layers=4,
        dim_ff=512,
        dropout=0.1,
    ).to(device)

    optimizer = AdamW(model.parameters(), lr=3e-4, weight_decay=0.01)

    target_slot_idx = slot_cols.index(role_slot_name)

    best_val_loss = float("inf")
    best_metrics  = None

    for epoch in range(1, num_epochs + 1):
        print(f"\nRole {role_slot_name} - Epoch {epoch}/{num_epochs}")

        train_loss = train_one_epoch(model, train_loader, optimizer, target_slot_idx)
        val_metrics = evaluate(model, val_loader, target_slot_idx)

        print(f"Train loss: {train_loss:.4f}")
        print(
            "Val loss: {loss:.4f} | top1: {top1:.4f} | top3: {top3:.4f} | top5: {top5:.4f}".format(
                loss=val_metrics['loss'],
                top1=val_metrics['top1_acc'],
                top3=val_metrics['top3_acc'],
                top5=val_metrics['top5_acc'],
            )
        )

        # simple best model tracking by val loss
        if val_metrics["loss"] < best_val_loss:
            best_val_loss = val_metrics["loss"]
            best_metrics = val_metrics
            torch.save(model.state_dict(), f"draft_model_{role_slot_name}.pt")

    results[role_slot_name] = best_metrics

print("\n=== Summary of best validation metrics per role ===")
for role, m in results.items():
    print(
        f"{role}: loss={m['loss']:.4f}, "
        f"top1={m['top1_acc']:.4f}, top3={m['top3_acc']:.4f}, top5={m['top5_acc']:.4f}"
    )



=== Training model for role: ally_TOP ===

Role ally_TOP - Epoch 1/10


Train loss: 4.0545
Val loss: 3.9454 | top1: 0.0886 | top3: 0.1979 | top5: 0.2793

Role ally_TOP - Epoch 2/10


Train loss: 3.9091
Val loss: 3.9027 | top1: 0.0948 | top3: 0.2093 | top5: 0.2925

Role ally_TOP - Epoch 3/10


Train loss: 3.8488
Val loss: 3.8835 | top1: 0.0946 | top3: 0.2096 | top5: 0.2949

Role ally_TOP - Epoch 4/10


Train loss: 3.8005
Val loss: 3.8819 | top1: 0.0946 | top3: 0.2122 | top5: 0.2964

Role ally_TOP - Epoch 5/10


...

Train loss: 3.3176
Val loss: 3.5076 | top1: 0.1304 | top3: 0.2832 | top5: 0.3894

Role ally_UTILITY - Epoch 7/10


Train loss: 3.2693
Val loss: 3.5200 | top1: 0.1271 | top3: 0.2776 | top5: 0.3863

Role ally_UTILITY - Epoch 8/10


Train loss: 3.2186
Val loss: 3.5515 | top1: 0.1241 | top3: 0.2697 | top5: 0.3773

Role ally_UTILITY - Epoch 9/10


Train loss: 3.1634
Val loss: 3.5887 | top1: 0.1253 | top3: 0.2718 | top5: 0.3777

Role ally_UTILITY - Epoch 10/10


Train loss: 3.1081
Val loss: 3.6265 | top1: 0.1197 | top3: 0.2658 | top5: 0.3721

=== Summary of best validation metrics per role ===
ally_TOP: loss=3.8819, top1=0.0946, top3=0.2122, top5=0.2964
ally_JUNGLE: loss=3.7454, top1=0.0821, top3=0.1993, top5=0.2904
ally_MIDDLE: loss=3.9156, top1=0.0838, top3=0.1919, top5=0.2775
ally_BOTTOM: loss=3.0521, top1=0.1874, top3=0.3636, top5=0.4824
ally_UTILITY: loss=3.4721, top1=0.1347, top3=0.2899, top5=0.3971
